In [235]:
%load_ext autoreload
%autoreload 2
from backtest.repository.webrepo.bithumb_repo import BithumbRepo
from backtest.repository.webrepo.upbit_repo import UpbitRepo
from backtest.repository.webrepo.binance_repo import BinanceRepo
from backtest.use_cases.stockdata_from_repo import stockdata_from_repo
from backtest.request.stockdata_from_repo import build_stock_data_from_repo_request
from backtest.domains.strategy import Strategy
from backtest.domains.strategy_result import StrategyResult,StrategyResultColumnType
from backtest.domains.backtest import Backtest
from backtest.use_cases.backtest_execute import backtest_execute

import pandas as pd

def basic_function(data: StockData, weight: int,name : str):
    response = StrategyResult(value=pd.DataFrame(
        index=data.data.index, columns=[name]))
    response.value[name] = [(
        StrategyResultColumnType.SELL, weight)] * len(data)
    return response

def basic_function2(data: StockData, weight: int,name : str):
    response = StrategyResult(value=pd.DataFrame(
        index=data.data.index, columns=[name]))
    """
    strategyfunction here
    """
    response.value[name] = [(
        StrategyResultColumnType.BUY, weight)] * len(data)
    return response



def rsi_function(data: StockData, weight: int,name : str,rolling=100):
    response = StrategyResult(value=pd.DataFrame(
        index=data.data.index, columns=[name]))
    """
    strategyfunction here
    """
    stockdata_raw = data.data
    stockdata_raw['sma'] = stockdata_raw['close'].rolling(rolling).mean().fillna(0)
    stockdata_raw['smashift'] = stockdata_raw['sma'].shift(1).fillna(0)
    def _rsi_internal(r):
        if (r.smashift - r.sma) > 0:
            return (StrategyResultColumnType.SELL, weight)
        elif (r.smashift - r.sma) == 0:
            return (StrategyResultColumnType.KEEP, weight)
        else:
            return (StrategyResultColumnType.BUY, weight)
    stockdata_raw['result'] = stockdata_raw.apply(lambda r :_rsi_internal(r),axis=1)

    response.value[name] = stockdata_raw['result']
    return response


request = build_stock_data_from_repo_request(
    filters={'order__eq': 'XRP', 'from__eq': '2021-01-01'})
response = stockdata_from_repo(BithumbRepo(), request=request)
stockdata = response.value
strategy = Strategy(name='rsi_function', function=rsi_function,weight=30,options={'rolling':100})
backtest = Backtest(strategy_list=[strategy], stockdata_list=[stockdata])
backtest_result = backtest_execute(backtest).value


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/sh4rk/Programming/Python/backtest/backtest/use_cases/backtest_execute.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stockdata_raw['total_profit'][index] = sell_profit


In [236]:
backtest_result.value['total_profit'].expanding().sum()

date
2021-01-01     0.000000
2021-01-02     0.000000
2021-01-03     0.000000
2021-01-04     0.000000
2021-01-05     0.000000
                ...    
2023-02-17   -50.750079
2023-02-18   -50.750079
2023-02-19   -50.750079
2023-02-20   -50.750079
2023-02-21   -50.750079
Freq: D, Name: total_profit, Length: 782, dtype: float64

In [237]:
pd.options.display.max_colwidth = 1000


backtest_result.value.tail(20)

,total_profit,stock_bucket
date,,
2023-02-02,0.0,[]
2023-02-03,0.0,[]
2023-02-04,0.0,[]
2023-02-05,0.0,[]
2023-02-06,0.0,[]
2023-02-07,0.0,[]
2023-02-08,0.0,[]
2023-02-09,0.0,[]
2023-02-10,0.0,[]
